In [3]:
# import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import shutil
# import cv2
from scipy import ndimage
# from skimage.transform import resize
from joblib import Parallel, delayed
import tensorflow as tf
from sklearn.model_selection import KFold
from tqdm.auto import tqdm

In [4]:
# get the names of the models in the model folder
model_names = ['model_1.h5', 'model_2.h5']

In [5]:
# how many models?
len(model_names)

2

In [6]:
model_names[0]

'model_1.h5'

In [7]:
loaded_models = []
for model in model_names:
    model_ld = tf.keras.models.load_model(r"C:\Users\15739\Desktop\UNL - all\Fall 2022\Manuscript\Filtered_90\codes_and_models" + "\\" + model)
    loaded_models.append(model_ld)

In [8]:
len(loaded_models)

2

In [9]:
# names of the test files
numpy_folder = r"C:\Users\15739\Desktop\UNL - all\Fall 2022\Manuscript\Filtered_90\np_test_filtered90"
test_files = os.listdir(numpy_folder)

In [10]:
# how many test files
len(test_files)

65

In [11]:
# Function to get the predictions given model and image

numpy_folder = r"C:\Users\15739\Desktop\UNL - all\Fall 2022\Manuscript\Filtered_90\np_test_filtered90"
v_stack_folder = r"C:\Users\15739\Desktop\UNL - all\Fall 2022\Manuscript\Filtered_90\test_np_stacks_hcc"

def prediction_on_test_data(model, selected_file, stride = 8, kernel_size = 32):
#     load the cnn model
    
# load the image data file
    load_image = np.load(numpy_folder + "\\"+ selected_file)
    
    # get the image height
    img_height = load_image.shape[0]
    # get the image weight
    img_width = load_image.shape[1]

    all_test_sub_windows = np.load(v_stack_folder + "\\"+ selected_file)

    # now, to get the predictions, pass the sub windows
    with tf.device('/CPU:0'):
        test_image_prediction = model.predict(all_test_sub_windows, batch_size = 100)
    
    # density map
    Density_map = np.zeros((img_height, img_width))

    # counts map
    counts_map = np.zeros((img_height, img_width))
    
    # now, for every window, we will keep adding the values together and also add the counts
    counter = 0
#     need a counter to move into each predicted value in the pred values list
    for ii in range(0, img_height, stride):
        for jj in range(0, img_width, stride):
#         operations for density map
#             get the window of interest
            new_window = Density_map[ii:ii + kernel_size,jj:jj+kernel_size]
#     fill each with the value c_k
            counts_window = np.full((new_window.shape[0], new_window.shape[1]), test_image_prediction[counter])
#     get the shapes of this new window
            cw_height = counts_window.shape[0]
            cw_width = counts_window.shape[1]
#         Do c_k/r_2
            counts_window_new = counts_window/(cw_height*cw_width)
#     This is the value in the window now
            value_window = counts_window_new
#     place the values in the corrsponding area of the density map
            Density_map[ii:ii + kernel_size,jj:jj+kernel_size] = new_window + value_window

#         Let's now focus on capturing the counts of the windows
            new_window_c = counts_map[ii:ii + kernel_size,jj:jj+kernel_size]
#     get the counts area
            count = np.ones((new_window_c.shape[0], new_window_c.shape[1]))
#     keep adding the counts to reflect the addition of densities
            counts_map[ii:ii + kernel_size,jj:jj+kernel_size] = new_window_c + count
#     increase the counter
            counter = counter + 1
            
#         get the normalized count
    normalized_counts = np.divide(Density_map, counts_map)
    
#     entire count on the test set
    pred_on_test = np.sum(normalized_counts)
    
#     return the predicted value
    return(pred_on_test, normalized_counts, selected_file)


In [19]:
# get the preds and normalized counts

all_preds_loop = []
for mdl in tqdm(loaded_models):
    preds_sub = []
    for file in test_files:
        results = prediction_on_test_data(mdl, file)
        preds_sub.append(results)
    all_preds_loop.append(preds_sub)

  0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
len(preds_sub)

65

In [146]:
# Extract the preds, image names, and density maps
all_dense_maps = []
all_preds = []
all_im_names = []
for i in all_preds_loop:
    dense_maps = []
    preds = []
    names = []
    for j in i:
        pred = j[0]
        preds.append(pred)
        name = j[2]
        names.append(name)
        dense_map = j[1]
        dense_maps.append(dense_map)
    all_dense_maps.append(dense_maps)
    all_preds.append(preds)
    all_im_names.append(names)
    


In [147]:
all_dfs = []
for i in range(0, len(all_dense_maps)):
    list_1 = all_im_names[i]
    list_2 = all_preds[i]
    df = pd.DataFrame(list(zip(list_1, list_2)), columns = ["Image_name", "Predicted_count"])
#     remove the image name
    all_dfs.append(df)

In [148]:
all_dfs

[                                           Image_name  Predicted_count
 0   190812_djifc6310s_Aug12_NW_row_32_range_6_rep_...        31.716618
 1   190812_djifc6310s_Aug12_SE_row_21_range_3_rep_...        23.549171
 2   190812_djifc6310s_Aug12_SE_row_32_range_4_rep_...        25.550615
 3   190812_djifc6310s_Aug12_SW_row_11_range_2_rep_...        33.694989
 4   190814_djifc6310s_Aug14_NE_row_33_range_2_rep_...        37.640023
 ..                                                ...              ...
 60  190905_djifc6310s_Sept5_SE_row_12_range_6_rep_...        30.788614
 61  190905_djifc6310s_Sept5_SW_row_11_range_2_rep_...        30.021164
 62  190905_djifc6310s_Sept5_SW_row_31_range_2_rep_...        51.282161
 63  190905_djifc6310s_Sept5_SW_row_32_range_5_rep_...        37.025334
 64  190905_djifc6310s_Sept5_SW_row_34_range_4_rep_...        25.251861
 
 [65 rows x 2 columns],
                                            Image_name  Predicted_count
 0   190812_djifc6310s_Aug12_NW_row_32

In [149]:
try_df = pd.concat(all_dfs, axis = 1)

In [150]:
try_df_no_im_name = try_df.drop(columns = "Image_name")

In [152]:
try_df_im_name = try_df[["Image_name"]]

In [153]:
try_df_im_name = pd.DataFrame(try_df_im_name.iloc[:,0])

In [154]:
all_preds = pd.concat((try_df_im_name, try_df_no_im_name),axis = 1)

In [155]:
all_preds.head()

,Image_name,Predicted_count,Predicted_count
0,190812_djifc6310s_Aug12_NW_row_32_range_6_rep_...,31.716618,31.538964
1,190812_djifc6310s_Aug12_SE_row_21_range_3_rep_...,23.549171,23.316486
2,190812_djifc6310s_Aug12_SE_row_32_range_4_rep_...,25.550615,26.272617
3,190812_djifc6310s_Aug12_SW_row_11_range_2_rep_...,33.694989,33.930508
4,190814_djifc6310s_Aug14_NE_row_33_range_2_rep_...,37.640023,37.797137


In [156]:
# add increments to the predicted counts
col_names = []
counter = 0
for i in all_preds.columns:
    name = i + '_' + str(counter)
    counter = counter + 1
    col_names.append(name)

In [157]:
# replace the old col names with new col names
all_preds.columns = col_names

In [158]:
all_preds.head()

,Image_name_0,Predicted_count_1,Predicted_count_2
0,190812_djifc6310s_Aug12_NW_row_32_range_6_rep_...,31.716618,31.538964
1,190812_djifc6310s_Aug12_SE_row_21_range_3_rep_...,23.549171,23.316486
2,190812_djifc6310s_Aug12_SE_row_32_range_4_rep_...,25.550615,26.272617
3,190812_djifc6310s_Aug12_SW_row_11_range_2_rep_...,33.694989,33.930508
4,190814_djifc6310s_Aug14_NE_row_33_range_2_rep_...,37.640023,37.797137


In [159]:
# Import the true counts csv file
true_counts = pd.read_csv(r"C:\Users\15739\Desktop\UNL - all\Fall 2022\Manuscript\Filtered_90\codes_and_models\true_counts_test_only_sample20.csv")

In [160]:
true_counts.head()

,Image_name,True_count
0,190812_djifc6310s_Aug12_NW_row_32_range_6_rep_...,53
1,190812_djifc6310s_Aug12_SE_row_21_range_3_rep_...,21
2,190812_djifc6310s_Aug12_SE_row_32_range_4_rep_...,19
3,190812_djifc6310s_Aug12_SW_row_11_range_2_rep_...,33
4,190814_djifc6310s_Aug14_NE_row_33_range_2_rep_...,50


In [161]:
true_count_col = true_counts['True_count']

In [162]:
all_preds

,Image_name_0,Predicted_count_1,Predicted_count_2
0,190812_djifc6310s_Aug12_NW_row_32_range_6_rep_...,31.716618,31.538964
1,190812_djifc6310s_Aug12_SE_row_21_range_3_rep_...,23.549171,23.316486
2,190812_djifc6310s_Aug12_SE_row_32_range_4_rep_...,25.550615,26.272617
3,190812_djifc6310s_Aug12_SW_row_11_range_2_rep_...,33.694989,33.930508
4,190814_djifc6310s_Aug14_NE_row_33_range_2_rep_...,37.640023,37.797137
...,...,...,...
60,190905_djifc6310s_Sept5_SE_row_12_range_6_rep_...,30.788614,30.866648
61,190905_djifc6310s_Sept5_SW_row_11_range_2_rep_...,30.021164,30.401564
62,190905_djifc6310s_Sept5_SW_row_31_range_2_rep_...,51.282161,52.057098
63,190905_djifc6310s_Sept5_SW_row_32_range_5_rep_...,37.025334,37.792152


In [164]:
# get the substracted values from each column
preds_only_df = all_preds.drop('Image_name_0', axis = 1)

In [165]:
subtracted_cols = abs(preds_nly_df.sub(true_count_col, axis = 0))

In [166]:
subtracted_cols.mean()

Predicted_count_1    8.920771
Predicted_count_2    9.443604
dtype: float64

In [167]:
0.5 * (8.920771 + 9.443604)

9.182187500000001

In [168]:
mean_all = subtracted_cols.mean().mean()
mean_all

9.182187599413034

In [169]:
std_all = subtracted_cols.mean().std()

In [170]:
lower_bnd = mean_all - 1.96*(std_all/np.sqrt(65))

In [171]:
lower_bnd

9.092310731530011

In [172]:
upper_bnd = mean_all + 1.96*(std_all/np.sqrt(65))

In [174]:
upper_bnd

9.272064467296056

In [178]:
# sanity check to see if the column names in the two dataframes are the same or not
all_names_only = []
for i in true_counts['Image_name']:
    name = i.split('.')[0]
    name_new = name + '.npy'
    all_names_only.append(name_new)
    
# get the names from the other df
names_list = all_preds["Image_name_0"].to_list()

In [181]:
# all names same?
all_names_only == names_list

True